# MIE1624 Assignment 1

Import Libraries


In [44]:
import numpy as np
import pandas as pd
import re
import html

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



## 1. Data Cleaning

Load files from current directory

In [ ]:
stops = open("stop_words.txt","r")
stopWords = stops.read().replace('\t','\n').split('\n')
airlineTweets = pd.read_csv("US_airline_tweets.csv")
genericTweets = pd.read_csv("generic_tweets.txt")

### Define data cleaning methods:

In [ ]:
#Method 1 - Remove all non ASCII charaters such as 'å'
def non_ascii_rm(text):
    allAscii = text.encode("ascii", errors="ignore").decode()
    return allAscii
    
#Method 2 - Remove all html tags and attributes
def html_tag_rm(text):
    noTag = re.compile(r"<.*?>").sub("", text)
    return noTag

#Method 3 - Replace Html character codes with ASCII equivalent
def ascii_rpl(text):
    return(html.unescape(text))

#Method 4 - Remove all URLs
def url_rm(text):
    return(re.compile(r"\w+://\S*").sub("", text))

#Method 5 - Remove all stopwords - loaded from stopword.txt
def stop_words_rm(stopWords,text):
    filtered_words = [word for word in text.split(" ") if word not in stopWords]
    return filtered_words

#Wraper function to preform all data cleaning method
def cleanText(text):
    global stopWords
    allAscii = non_ascii_rm(text.lower()) #make sure all lowercase
    noTag = html_tag_rm(allAscii) 
    asciiRpl = ascii_rpl(noTag)
    noUrl = url_rm(asciiRpl)
    noStopWords = stop_words_rm(stopWords, noUrl)
    return(" ".join(noStopWords))

Clean Data using methods defined above

In [ ]:
cleandata = lambda x: cleanText(x)
genericTweets["text"] = genericTweets["text"].map(cleandata)
airlineTweets["text"] = airlineTweets["text"].map(cleandata)

genericTweets.to_csv("generic_tweets_clean.csv")
airlineTweets.to_csv("US_airline_tweets_clean.csv")

## 2. Exploratory Analysis

In [ ]:
atUser = pd.DataFrame(columns=['atUser'])
hashTag = pd.DataFrame(columns=['hashTag'])

#Method to find regex pattern - used for finding "@"s and "#"s
def getAtUsers(text, pattern):
    return pattern.findall(text)

#find airlines mentioned with "@"
atUserRe = re.compile(r'@\w+')
for t in airlineTweets["text"]:
    for user in getAtUsers(t, atUserRe):
        atUser = atUser.append({"atUser": user},ignore_index=True)
        
atAirCount = atUser.groupby(["atUser"]).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False).head(20)
atAirCount

In [ ]:
import matplotlib.pyplot as plt
#atAirCount = atAirCount.set_index('atUser')
aa = atAirCount.set_index("atUser")
aa.index.name="airline"
plot = aa.head(7).plot(kind="bar",alpha=0.5, title="airline tweet count")
plot.set_ylabel("counts")


In [2]:
genClean = airlineTweets = pd.read_csv("generic_tweets_clean.csv")
airClean = genericTweets = pd.read_csv("US_airline_tweets_clean.csv")



In [3]:
def deepClean(text):
    text = str(text)
    #remove @
    text = re.sub(r'@\S+', '', text)
    #remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    return text

#airDeepClean = airClean.head(10000)
#for c in range(airDeepClean.text.size):
#    if airDeepClean.text[c]:
#        airDeepClean.text[c] = deepClean(airDeepClean.text[c])

#genDeepClean = genClean.head(10000)
#for c in range(genDeepClean.text.size):
#    genDeepClean.text[c] = deepClean(genDeepClean.text[c])
print(airClean.shape)
print(genClean.shape)
cleandata = lambda x: deepClean(x)
genClean["text"] = genClean["text"].map(cleandata)
airClean["text"] = airClean["text"].map(cleandata)


    

(11541, 7)
(200000, 7)


In [48]:
g_sample = genClean
air_sample = airClean

cv = CountVectorizer(binary=True)
cv.fit(g_sample.text)
X = cv.transform(g_sample.text)
target = g_sample["class"]

X_test = cv.transform(air_sample.text)
X_test_target = [4 if s == "possitive" else 0 for s in air_sample.sentiment]


X_train, X_val, y_train, y_val = train_test_split(X, target , test_size=0.3, random_state=42)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c, solver="saga")
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

final_model = LogisticRegression(C=0.5)
final_model.fit(X, target)

print ("Final Accuracy: %s" 
       % accuracy_score(X_test_target, final_model.predict(X_test)))    
    
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)

print("---------")

for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

Accuracy for C=0.01: 0.7237666666666667
Accuracy for C=0.05: 0.737
Accuracy for C=0.25: 0.7421666666666666
Accuracy for C=0.5: 0.7425
Accuracy for C=1: 0.7403
Final Accuracy: 0.6236028073823758
('followfriday', 1.9673256401549069)
('welcome', 1.9524867418965373)
('yayy', 1.8481230527703165)
('congratulations', 1.8480105618454274)
('smiling', 1.828590532659728)
('thankyou', 1.757862522452095)
('smile', 1.7228350886004498)
('blessed', 1.700541935593226)
('pleasure', 1.6867064262637803)
('refreshed', 1.6441721076018658)
---------
('sad', -3.1219590264864525)
('sadly', -2.9062153656944667)
('bummed', -2.6771705389101497)
('mothers', -2.650108958004124)
('gutted', -2.49139775091238)
('poor', -2.4804936480328097)
('lonely', -2.408432628846069)
('missing', -2.379144617324849)
('misses', -2.3257595316754407)
('hates', -2.3145085727652175)


In [56]:
dfReasons = airClean.query('negative_reason == negative_reason').copy()
#reasons = dfReasons.negative_reason.unique()
print(dfReasons.shape)
print(le.fit_transform(dfReasons["negative_reason"]).shape)

le = preprocessing.LabelEncoder()
dfReasons["negative_reason"] = le.fit_transform(dfReasons["negative_reason"])


cv = CountVectorizer(binary=True)
cv.fit(dfReasons.text)
X = cv.transform(dfReasons.text)
target = dfReasons.negative_reason

X_train, X_val, y_train, y_val = train_test_split(X, target , test_size=0.3, random_state=42)

final_model = LogisticRegression(C=0.5, solver='lbfgs',
                         multi_class='multinomial')
final_model.fit(X_train, y_train)

print ("Final Accuracy: %s" 
       % accuracy_score(y_val, final_model.predict(X_val)))



(9178, 7)
(9178,)
Final Accuracy: 0.6161946259985476


862      9
6818     7
2670     3
8380     1
8738     1
2956     3
4159     3
7647     3
9672     1
2473     5
2552     3
6006     7
7203     3
7644     3
7890     3
9440     1
4669     3
7810     3
9418     3
11375    1
11384    3
2309     7
3485     3
998      0
10001    1
10553    3
6830     1
6990     3
5853     8
8131     3
        ..
975      3
8550     7
10623    2
382      3
10693    2
9831     7
5100     8
9175     3
3804     3
3525     1
6687     7
6896     1
7837     3
7705     5
6821     7
2959     2
5601     0
7188     2
2443     1
8902     6
1918     1
10600    8
7307     8
5562     3
8865     3
7332     7
2065     6
3939     1
10065    3
1174     4
Name: negative_reason, Length: 2754, dtype: int64